In [1]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

df = pdr.get_data_yahoo("IBM", start="2019-01-01", end="2024-01-01")

[*********************100%%**********************]  1 of 1 completed


In [7]:
reversed_df = df.iloc[::-1]


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features_fwd = scaler.fit_transform(df) 
scaled_features_bkwd = scaler.fit_transform(reversed_df) 

In [14]:
import numpy as np

def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length][4] 
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

seq_length = 5
X_fwd, y_fwd = create_sequences(scaled_features_fwd, seq_length)
X_bkwd, y_bkwd = create_sequences(scaled_features_bkwd, seq_length)

In [18]:
split_fraction = 0.8
split = int(split_fraction * len(X_fwd))

X_train_fwd, X_test_fwd = X_fwd[:split], X_fwd[split:]
y_train_fwd, y_test_fwd = y_fwd[:split], y_fwd[split:]

X_train_bkwd, X_test_bkwd = X_bkwd[:split], X_bkwd[split:]
y_train_bkwd, y_test_bkwd = y_bkwd[:split], y_bkwd[split:]

In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64  

#Forward
train_data = TensorDataset(torch.Tensor(X_train_fwd), torch.Tensor(y_train_fwd))
train_loader_fwd = DataLoader(train_data, shuffle=True, batch_size=batch_size)

test_data = TensorDataset(torch.Tensor(X_test_fwd), torch.Tensor(y_test_fwd))
test_loader_fwd = DataLoader(test_data, shuffle=False, batch_size=batch_size)

#Backward
train_data = TensorDataset(torch.Tensor(X_train_bkwd), torch.Tensor(y_train_bkwd))
train_loader_bkwd = DataLoader(train_data, shuffle=True, batch_size=batch_size)

test_data = TensorDataset(torch.Tensor(X_test_bkwd), torch.Tensor(y_test_bkwd))
test_loader_bkwd = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [20]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out

## Forward Training

In [28]:
import torch.optim as optim

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    print("Running on GPU")
else: print("Running on CPU")

#Model Define
model = SimpleRNN(input_size=6, hidden_size=20, output_size=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
model.train()

for epoch in range(num_epochs):
    for inputs, labels in train_loader_fwd:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels.unsqueeze(-1))
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Running on CPU
Epoch [1/100], Loss: 0.2538
Epoch [2/100], Loss: 0.1292
Epoch [3/100], Loss: 0.1371
Epoch [4/100], Loss: 0.0785
Epoch [5/100], Loss: 0.0678
Epoch [6/100], Loss: 0.0411
Epoch [7/100], Loss: 0.0225
Epoch [8/100], Loss: 0.0406
Epoch [9/100], Loss: 0.0206
Epoch [10/100], Loss: 0.0154
Epoch [11/100], Loss: 0.0132
Epoch [12/100], Loss: 0.0242
Epoch [13/100], Loss: 0.0080
Epoch [14/100], Loss: 0.0171
Epoch [15/100], Loss: 0.0285
Epoch [16/100], Loss: 0.0129
Epoch [17/100], Loss: 0.0151
Epoch [18/100], Loss: 0.0085
Epoch [19/100], Loss: 0.0212
Epoch [20/100], Loss: 0.0166
Epoch [21/100], Loss: 0.0447
Epoch [22/100], Loss: 0.0111
Epoch [23/100], Loss: 0.0318
Epoch [24/100], Loss: 0.0172
Epoch [25/100], Loss: 0.0120
Epoch [26/100], Loss: 0.0120
Epoch [27/100], Loss: 0.0159
Epoch [28/100], Loss: 0.0138
Epoch [29/100], Loss: 0.0149
Epoch [30/100], Loss: 0.0149
Epoch [31/100], Loss: 0.0185
Epoch [32/100], Loss: 0.0115
Epoch [33/100], Loss: 0.0204
Epoch [34/100], Loss: 0.0290
Epoch [3

## Backward Training

In [30]:
import torch.optim as optim

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    print("Running on GPU")
else: print("Running on CPU")

#Model Define
model = SimpleRNN(input_size=6, hidden_size=20, output_size=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
model.train()

for epoch in range(num_epochs):
    for inputs, labels in train_loader_bkwd:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels.unsqueeze(-1))
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Running on CPU
Epoch [1/100], Loss: 1.0330
Epoch [2/100], Loss: 0.5794
Epoch [3/100], Loss: 0.2865
Epoch [4/100], Loss: 0.1555
Epoch [5/100], Loss: 0.2104
Epoch [6/100], Loss: 0.1575
Epoch [7/100], Loss: 0.1279
Epoch [8/100], Loss: 0.1160
Epoch [9/100], Loss: 0.0768
Epoch [10/100], Loss: 0.0934
Epoch [11/100], Loss: 0.0569
Epoch [12/100], Loss: 0.0883
Epoch [13/100], Loss: 0.0724
Epoch [14/100], Loss: 0.0359
Epoch [15/100], Loss: 0.0232
Epoch [16/100], Loss: 0.0167
Epoch [17/100], Loss: 0.0211
Epoch [18/100], Loss: 0.0284
Epoch [19/100], Loss: 0.0151
Epoch [20/100], Loss: 0.0099
Epoch [21/100], Loss: 0.0182
Epoch [22/100], Loss: 0.0163
Epoch [23/100], Loss: 0.0276
Epoch [24/100], Loss: 0.0079
Epoch [25/100], Loss: 0.0184
Epoch [26/100], Loss: 0.0119
Epoch [27/100], Loss: 0.0102
Epoch [28/100], Loss: 0.0133
Epoch [29/100], Loss: 0.0135
Epoch [30/100], Loss: 0.0092
Epoch [31/100], Loss: 0.0133
Epoch [32/100], Loss: 0.0257
Epoch [33/100], Loss: 0.0090
Epoch [34/100], Loss: 0.0209
Epoch [3